# Generate:

## Imports

In [1]:
import tensorflow as tf
import os
import numpy as np
import midiutil
import pandas as pd
import sklearn
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from tensorflow import keras
from midiutil import MIDIFile
from os import listdir

import warnings
warnings.filterwarnings('ignore')

## Load Models

In [2]:
m_n_path = os.path.join(os.getcwd(),'Models/Curves/3 Layer/m_n.h5')
m_e_path = os.path.join(os.getcwd(),'Models/Curves/3 Layer/m_e.h5')
m_h_path = os.path.join(os.getcwd(),'Models/Curves/3 Layer/m_h.h5')

m_n=keras.models.load_model(m_n_path)
m_e=keras.models.load_model(m_e_path)
m_h=keras.models.load_model(m_h_path)

## Load Input/Output

In [9]:
in_file_path= os.path.join(os.getcwd(), 'Data/Processed/', 'input.txt')
out_file_path= os.path.join(os.getcwd(), 'Data/Processed/', 'output.txt')

inp = pd.read_csv(in_file_path,sep=",",header=None)
out = pd.read_csv(out_file_path,sep=",",header=None)

out_save = out.copy()

In [10]:
out_fixed=out.copy()
out_fixed=out_fixed.astype(np.int)
out =pd.DataFrame(out_fixed)

In [11]:
count_notes = np.sum(out,axis=1)
count_notes_hot = pd.get_dummies(count_notes)

## Predict

In [12]:
def postprocess(y_pred,tbin,):
    note_info=np.zeros((1,3))
    y_copy = y_pred.copy()
    time=0
    time_len = np.diff(tbin)[1]
    for i in range(tbin.shape[0]):
        if (i==0):
            time=0
        elif (i==1):
            time = time+np.diff(tbin)[0]
        else:
            time=time+time_len
        
        notes_played=np.where(y_copy[i,:]==1)[0]
        for j in range(notes_played.shape[0]):
            note_info_temp=np.zeros((1,3))
            note_info_temp[0,0]=notes_played[j]
            note_info_temp[0,1]=time
            k=0
            if (i==tbin.shape[0]-1):
                break
            while (y_copy[i+k,notes_played[j]]==1):
                if (i+k==tbin.shape[0]-1):
                    break
                k=k+1
                note_info_temp[0,2]=note_info_temp[0,2]+1
            for l in range(k+1):
                y_copy[i+l,notes_played[j]]=0
            note_info=np.append(note_info,note_info_temp,axis=0)
    return note_info

In [13]:
dir_path = 'Data/Processed/Testing/'
dir_path1 = 'Data/Processed/Testing/'

In [16]:
input_dir_path_in=os.path.join(os.getcwd(),dir_path1,'input')
tbin_dir_path_in=os.path.join(os.getcwd(),dir_path1,'tbin')

input_file = listdir(input_dir_path_in)
tbin_files = listdir(tbin_dir_path_in)
for i in range(len(input_file)):
    inp_test_path = os.path.join(os.getcwd(),dir_path1,'input',input_file[i])
    inp_test=pd.read_csv(inp_test_path,sep=',',header=None)
    print(inp_test_path)
    y_pred_n = m_n.predict(inp_test,batch_size=8)
    y_pred_e = m_e.predict(inp_test,batch_size=8)
    y_pred_h = m_h.predict(inp_test,batch_size=8)
    
    tbin_path = os.path.join(os.getcwd(),dir_path1,'tbin',tbin_files[i])
    tbin=pd.read_csv(tbin_path,sep=',',header=None)
    init_tbin=np.zeros(1)
    tbin=np.append(init_tbin,tbin)
    
    time_len = np.diff(tbin)[1]
    
    y_note_pred_n = np.zeros((y_pred_n.shape[0],y_pred_n.shape[1]))
    for j in range(y_pred_n.shape[0]):
        out_probs=np.zeros((88,1))
        out_probs[:,0]=y_pred_n[j,:].copy()
        out_probs[out_probs<0.03]=0
        kmeans=KMeans(n_clusters=2).fit(out_probs)
        threshold = np.maximum(kmeans.cluster_centers_[0],kmeans.cluster_centers_[1])
        if threshold==0:
            threshold = 1
        out_probs=y_pred_n[j,0:88].copy()
        out_probs[out_probs>=threshold]=1
        out_probs[out_probs<threshold]=0
        y_note_pred_n[j,:]=out_probs
    
    track    = 0
    channel  = 0
    time     = 0   
    duration = 1   
    tempo    = 60 
    volume   = 100 
    MyMIDI_n = MIDIFile(1)
    MyMIDI_n.addTempo(track, time, tempo)
    
    processed_out=postprocess(y_note_pred_n,tbin)
            
    processed_out[:,0]=processed_out[:,0]+21
    processed_out=processed_out[1:processed_out.shape[0],:]
    for j in range(processed_out.shape[0]):
        MyMIDI_n.addNote(track,channel,processed_out[j,0].astype(np.int),processed_out[j,1],processed_out[j,2]*time_len,volume)
    
    with open("out_"+input_file[i][:-4]+"_n.mid", "wb") as output_file:
        MyMIDI_n.writeFile(output_file)
        
    y_note_pred_e=np.zeros((y_pred_e.shape[0],88))
    
    for j in range(y_pred_e.shape[0]):
        out_probs = y_pred_e[j,:].copy()
        if (np.argmax(out_probs)==88):
            y_note_pred_e[j,:]=np.zeros(88)
        else:
            out_probs=np.zeros((88,1))
            out_probs[:,0]=y_pred_e[j,0:88].copy()
            out_probs[out_probs<0.03]=0
            kmeans1=KMeans(n_clusters=2).fit(out_probs)
            threshold = np.maximum(kmeans1.cluster_centers_[0],kmeans1.cluster_centers_[1])
            if threshold==0:
                threshold = 1
            out_probs=y_pred_e[j,0:88].copy()
            out_probs[out_probs>=threshold]=1
            out_probs[out_probs<threshold]=0
            y_note_pred_e[j,:]=out_probs
            
    
    track    = 0
    channel  = 0
    time     = 0 
    duration = 1   
    tempo    = 60  
    volume   = 100 
    MyMIDI_e = MIDIFile(1)
    MyMIDI_e.addTempo(track, time, tempo)
    
    processed_out=postprocess(y_note_pred_e,tbin)
    processed_out[:,0]=processed_out[:,0]+21
    processed_out=processed_out[1:processed_out.shape[0],:]
    for j in range(processed_out.shape[0]):
        MyMIDI_e.addNote(track,channel,processed_out[j,0].astype(np.int),processed_out[j,1],processed_out[j,2]*time_len,volume)
        
    with open("out_"+input_file[i][:-4]+"_e.mid", "wb") as output_file:
        MyMIDI_e.writeFile(output_file)
        
        
    threshold_pred_h = y_pred_h.copy()[:,0:88]
    num_pred_prob = y_pred_h.copy()
    
    for j in range(y_pred_h.shape[0]):
        out_probs = num_pred_prob[:,88:num_pred_prob.shape[1]]
        num_pred=count_notes_hot.columns[np.argmax(out_probs[j,:])]
        pred_idx = np.flip(np.argsort(threshold_pred_h[j,:]))[0:num_pred]
        out_pred=np.zeros(88)
        if (num_pred==0):
            threshold_pred_h[j,:]=out_pred
        else:
            out_pred[pred_idx]=1
            threshold_pred_h[j,:]=out_pred
        
    y_note_pred_h = threshold_pred_h[:,0:88]
    
    track    = 0
    channel  = 0
    time     = 0
    duration = 1  
    tempo    = 60 
    volume   = 100 
    MyMIDI_h = MIDIFile(1)
    MyMIDI_h.addTempo(track, time, tempo)
    
    processed_out=postprocess(y_note_pred_h,tbin)
    
    processed_out[:,0]=processed_out[:,0]+21
    processed_out=processed_out[1:processed_out.shape[0],:]
    for j in range(processed_out.shape[0]):
        MyMIDI_h.addNote(track,channel,processed_out[j,0].astype(np.int),processed_out[j,1],processed_out[j,2]*time_len,volume)
        
    with open("out_"+input_file[i][:-4]+"_h.mid", "wb") as output_file:
        MyMIDI_h.writeFile(output_file)
    print()
    print('Done with', input_file[i][:-4])
    
print('DONE')

/afs/inf.ed.ac.uk/user/s17/s1709221/IntTest/Automatic-Music-Transcription/Data/Processed/Testing/input/input.txt

Done with input
DONE
